# Prep variables for HydroTas 2020-2021 workplan:
- Skill assessment
  - rainfall, surface temp and surface wind over Australia region and Tasmania region
  - Assess skill as function of start month and ensemble size
- UNSEEN
  - Tasmanian rainfall and Melbourne surface temperature

In [114]:
import glob
import cftime
import geopandas
import regionmask
import numpy as np
import xarray as xr
import myfuncs as my
from dask.distributed import Client
from dask_jobqueue import PBSCluster

In [101]:
walltime = '01:00:00'
cores = 24
memory = '48GB'

cluster = PBSCluster(processes=1,
                     walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),
                                '-l mem='+str(memory),
                                '-P ux06',
                                '-l jobfs=100GB',
                                '-l storage=gdata/xv83+gdata/v14+scratch/v14'],
                     local_directory='$PBS_JOBFS',
                     header_skip=['select'])

In [102]:
cluster.scale(jobs=2)
client = Client(cluster)
client

Client Scheduler: tcp://10.6.73.35:38743 Dashboard: http://10.6.73.35:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Create Australia land mask on CAFE grid

In [115]:
def preprocess_f6_atmos(ds):
    """ Preprocess steps for the CAFE-f6 atmos forecasts"""
    # Drop some coordinates
    for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
        if drop_coord in ds.coords:
            ds = ds.drop(drop_coord)
    # Truncate latitudes to 10dp
    for dim in ds.dims:
        if 'lat' in dim:
            ds = ds.assign_coords({dim: ds[dim].round(decimals=10)})
    return ds

In [116]:
CAFE_area = preprocess_f6_atmos(
    my.open_zarr(
        '/g/data/v14/vxk563/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-20201101/ZARR/atmos_isobaric_daily.zarr.zip',
    )['area'])

NRM = geopandas.read_file('data/NRM_clusters/NRM_clusters.shp')
regions = regionmask.Regions_cls(
    name='NRM_regions', 
    numbers=list(NRM.index), 
    names=list(NRM.label), 
    abbrevs=list(NRM.code),
    outlines=list(NRM.geometry))
regions_mask = regions.mask(CAFE_area, lon_name='lon', lat_name='lat')

Australia_mask = xr.where(regions_mask.notnull(), True, False)

# Parameters

In [117]:
REGIONS = {'AUS': Australia_mask,
           'TAS': (-42, 146.5),
           'MEL': (-37.81, 144.96)}

# Get f6 atmospheric monthly variables

In [118]:
VARIABLES = {'precip': 
                 {'name': 'precip', 
                  'regions': ['AUS', 'TAS', 'MEL']},
             't_ref': 
                 {'name': 't_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'u_ref': 
                 {'name': 'u_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'v_ref': 
                 {'name': 'v_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'h500':
                 {'name': 'h500',
                  'regions': ['NATIVE']}}

In [119]:
paths_xv83 = glob.glob(
    '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-??????01/ZARR/atmos_isobaric_month.zarr.zip'
)
paths_v14 = glob.glob(
    '/g/data/v14/vxk563/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-??????01/ZARR/atmos_isobaric_month.zarr.zip'
)
paths = sorted(paths_xv83+paths_v14, key=lambda x: x.split('/')[-3])

In [120]:
%%time
ds = my.open_zarr_forecasts(
        paths, 
        variables=VARIABLES.keys(),
        preprocess=preprocess_f6_atmos
).rename(
    {k: v['name'] for k, v in VARIABLES.items()})

for v_ in VARIABLES:
    v = VARIABLES[v_]['name']
    var = ds[[v]]
    regions = VARIABLES[v_]['regions']
    print(f'Extracting {v_} over region(s) {regions}...')
    
    for r in regions:
        # Weighted mean over region
        if r == 'NATIVE':
            var_region = var
        else:
            var_region = my.get_region(
                var, REGIONS[r]).weighted(
                CAFE_area).mean(
                ['lat','lon'])
            
            # Chunk appropriately
            var_region = var_region.chunk({'init_date': -1, 'lead_time': -1})
        
        # Fill nans in time with dummy times so that time operations work nicely
        var_region = var_region.assign_coords({
            'time': var_region.time.fillna(cftime.DatetimeJulian(1800, 1, 1))})

        # Save
        my.to_zarr(var_region, f'./data/f6_{v}_{r}_raw.zarr')

Extracting precip over region(s) ['AUS', 'TAS', 'MEL']...
Extracting t_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting u_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting v_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting h500 over region(s) ['NATIVE']...
CPU times: user 1min 41s, sys: 40.8 s, total: 2min 22s
Wall time: 4min 21s


In [121]:
f6_lead_times = ds.lead_time
f6_init_dates = ds.init_date

# Get f6 ocean monthly variables

In [103]:
VARIABLES = {'sst': 
                 {'name': 'sst', 
                  'regions': ['NATIVE']}}

In [104]:
paths_xv83 = glob.glob(
    '/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-??????01/ZARR/ocean_month.zarr.zip'
)
paths_v14 = glob.glob(
    '/g/data/v14/vxk563/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-??????01/ZARR/ocean_month.zarr.zip'
)
paths = sorted(paths_xv83+paths_v14, key=lambda x: x.split('/')[-3])

In [105]:
# Remove 2000-05 forecast for now because it's not quite finished
paths.remove('/g/data/xv83/ds0092/CAFE/forecasts/f6/WIP/c5-d60-pX-f6-20000501/ZARR/ocean_month.zarr.zip')

In [106]:
def preprocess_f6_ocean(ds):
    """ Preprocess steps for the CAFE-f6 ocean forecasts"""
    # Drop some coordinates
    for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'geolat_t', 'geolon_t', 'area_t']:
        if drop_coord in ds.coords:
            ds = ds.drop(drop_coord)
    return ds

In [107]:
%%time
ds = my.open_zarr_forecasts(
        paths, 
        variables=VARIABLES.keys(),
        preprocess=preprocess_f6_ocean
).rename(
    {k: v['name'] for k, v in VARIABLES.items()})

for v_ in VARIABLES:
    v = VARIABLES[v_]['name']
    var = ds[[v]]
    regions = VARIABLES[v_]['regions']
    print(f'Extracting {v_} over region(s) {regions}...')
    
    for r in regions:
        # Weighted mean over region
        if r == 'NATIVE':
            var_region = var
        else:
            var_region = my.get_region(
                var, REGIONS[r]).weighted(
                CAFE_area).mean(
                ['lat','lon'])
            
            # Chunk appropriately
            var_region = var_region.chunk({'init_date': -1, 'lead_time': -1})
        
        # Fill nans in time with dummy times so that time operations work nicely
        var_region = var_region.assign_coords({
            'time': var_region.time.fillna(cftime.DatetimeJulian(1800, 1, 1))})

        # Save
        my.to_zarr(var_region, f'./data/f6_{v}_{r}_raw.zarr')

Extracting sst over region(s) ['NATIVE']...
CPU times: user 6min 13s, sys: 6min 46s, total: 13min
Wall time: 20min 16s


# Get f5 atmospheric monthly variables

In [122]:
VARIABLES = {'precip': 
                 {'name': 'precip', 
                  'regions': ['AUS', 'TAS', 'MEL']},
             't_ref': 
                 {'name': 't_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'u_ref': 
                 {'name': 'u_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'v_ref': 
                 {'name': 'v_ref',
                  'regions': ['AUS', 'TAS', 'MEL']},
             'h500':
                 {'name': 'h500',
                  'regions': ['NATIVE']}}

In [123]:
path = '/g/data/v14/vxk563/CAFE/forecasts/f5/ZARR/atmos_isobaric_month.zarr.zip'

In [124]:
def preprocess_f5(ds):
    """ Preprocess steps for the CAFE-f6 forecasts"""
    # Drop some coordinates
    for drop_coord in ['average_DT', 'average_T1', 'average_T2', 'zsurf', 'area']:
        if drop_coord in ds.coords:
            ds = ds.drop(drop_coord)
    # Truncate latitudes to 10dp
    for dim in ds.dims:
        if 'lat' in dim:
            ds = ds.assign_coords({dim: ds[dim].round(decimals=10)})
    return ds

In [126]:
%%time
ds = my.open_zarr(
        path, 
        variables=VARIABLES.keys(),
        preprocess=preprocess_f5).rename(
    {k: v['name'] for k, v in VARIABLES.items()})

for v_ in VARIABLES:
    v = VARIABLES[v_]['name']
    var = ds[[v]]
    regions = VARIABLES[v_]['regions']
    print(f'Extracting {v_} over region(s) {regions}...')
    
    for r in regions:
        # Weighted mean over region
        if r == 'NATIVE':
            var_region = var
        else:
            var_region = my.get_region(
                var, REGIONS[r]).weighted(
                CAFE_area).mean(
                ['lat','lon'])
            
            # Chunk appropriately
            var_region = var_region.chunk({'init_date': -1, 'lead_time': -1})
        
        # Fill nans in time with dummy times so that time operations work nicely
        var_region.time.attrs['calendar_type'] = 'JULIAN'
        var_region = var_region.assign_coords({
            'time': var_region.time.fillna(cftime.DatetimeJulian(1800, 1, 1))})

        # Save
        my.to_zarr(var_region, f'./data/f5_{v}_{r}_raw.zarr')

Extracting precip over region(s) ['AUS', 'TAS', 'MEL']...
Extracting t_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting u_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting v_ref over region(s) ['AUS', 'TAS', 'MEL']...
Extracting h500 over region(s) ['NATIVE']...


In [127]:
f5_lead_times = ds.lead_time
f5_init_dates = ds.init_date

In [128]:
obsv_lead_times = xr.concat([f5_lead_times, f6_lead_times], dim='lead_time')
obsv_lead_times = obsv_lead_times[np.unique(obsv_lead_times, return_index=True)[1]]

obsv_init_dates = xr.concat([f5_init_dates, f6_init_dates], dim='init_date')
obsv_init_dates = obsv_init_dates[np.unique(obsv_init_dates, return_index=True)[1]]

# JRA-55 surface data

In [177]:
VARIABLES = {'TPRAT_GDS0_SFC': 
                 {'name': 'precip', 
                  'regions': ['AUS', 'TAS', 'MEL']},
             'TMP_GDS0_HTGL': 
                 {'name': 't_ref', 
                  'regions': ['AUS', 'TAS', 'MEL']},
             'UGRD_GDS0_HTGL': 
                 {'name': 'u_ref', 
                  'regions': ['AUS', 'TAS', 'MEL']},
             'VGRD_GDS0_HTGL': 
                 {'name': 'v_ref', 
                  'regions': ['AUS', 'TAS', 'MEL']}}

In [178]:
path = '/g/data/v14/ds0092/data/ZARR/csiro-dcfp-jra55/surface_month_cafe-grid.zarr'

In [179]:
def preprocess_jra(ds):
    """ Preprocess steps for the JRA data"""
    # Rename time and level
    for key, val in {'initial_time0_hours': 'time', 
                     'lv_ISBL1': 'level'}.items():
        if key in ds.coords:
                ds = ds.rename({key: val})
    # Drop filename attribute
    del ds.attrs['filename']
    # Truncate latitudes to 10dp
    for dim in ds.dims:
        if 'lat' in dim:
            ds = ds.assign_coords({dim: ds[dim].round(decimals=10)})
    return ds

In [182]:
%%time
ds = my.open_zarr(
        path, 
        variables=VARIABLES.keys(), 
        preprocess=preprocess_jra).rename(
    {k: v['name'] for k, v in VARIABLES.items()})

for v_ in VARIABLES:
    v = VARIABLES[v_]['name']
    var = ds[[v]]
    regions = VARIABLES[v_]['regions']
    print(f'Extracting {v_} over region(s) {regions}...')
    
    for r in regions:
        # Weighted mean over region
        if r == 'NATIVE':
            var_region = var
        else:
            var_region = my.get_region(
                var, REGIONS[r]).weighted(
                CAFE_area).mean(
                ['lat','lon'])
            
            # Chunk appropriately
            var_region = var_region.chunk({'time': -1})
            
        # Stack by initial date
        var_stacked = my.stack_by_init_date(
            var_region, 
            obsv_init_dates, 
            len(obsv_lead_times)).chunk(
            {'init_date': -1, 'lead_time': -1})
        
        # Fill nans in time with dummy times so that time operations work nicely
        var_stacked.time.attrs['calendar_type'] = 'Proleptic_Gregorian'
        var_stacked = var_stacked.assign_coords({
            'time': var_stacked.time.fillna(cftime.DatetimeProlepticGregorian(1800, 1, 1))})

        # Save
        my.to_zarr(var_region, f'./data/jra55_{v}_{r}_ts.zarr')
        my.to_zarr(var_stacked, f'./data/jra55_{v}_{r}.zarr')

Extracting TPRAT_GDS0_SFC over region(s) ['AUS', 'TAS', 'MEL']...
Extracting TMP_GDS0_HTGL over region(s) ['AUS', 'TAS', 'MEL']...
Extracting UGRD_GDS0_HTGL over region(s) ['AUS', 'TAS', 'MEL']...
Extracting VGRD_GDS0_HTGL over region(s) ['AUS', 'TAS', 'MEL']...
CPU times: user 17.5 s, sys: 1.58 s, total: 19.1 s
Wall time: 37.3 s


# AWAP monthly data

In [188]:
VARIABLES = {'precip': 
                 {'name': 'precip', 
                  'regions': ['AUS', 'TAS', 'MEL']}}

In [189]:
path = '/g/data/v14/ds0092/data/ZARR/csiro-dcfp-csiro-awap/rain_day_19000101-20201202_cafe-grid.zarr'

In [190]:
def preprocess_awap(ds):
    """ Preprocess steps for the AWAP data"""
    # Truncate latitudes to 10dp
    for dim in ds.dims:
        if 'lat' in dim:
            ds = ds.assign_coords({dim: ds[dim].round(decimals=10)})
    return ds

In [ ]:
%%time
ds = my.open_zarr(
        path, 
        variables=VARIABLES.keys(), 
        preprocess=preprocess_awap).rename(
    {k: v['name'] for k, v in VARIABLES.items()})

# Sum to monthly values
def sum_min_samples(ds, dim, min_samples):
    """ Return sum only if there are more than min_samples along dim """
    s = ds.sum(dim, skipna=False)
    return s if len(ds[dim]) >= min_samples else np.nan*s
ds = ds.resample(time='MS').map(sum_min_samples, dim='time', min_samples=28)

for v_ in VARIABLES:
    v = VARIABLES[v_]['name']
    var = ds[[v]]
    regions = VARIABLES[v_]['regions']
    print(f'Extracting {v_} over region(s) {regions}...')
    
    for r in regions:
        # Weighted mean over region
        if r == 'NATIVE':
            var_region = var
        else:
            var_region = my.get_region(
                var, REGIONS[r]).weighted(
                CAFE_area).mean(
                ['lat','lon'])
            
            # Chunk appropriately
            var_region = var_region.chunk({'time': -1})
            
        # Stack by initial date
        var_stacked = my.stack_by_init_date(
            var_region, 
            obsv_init_dates, 
            len(obsv_lead_times)).chunk(
            {'init_date': -1, 'lead_time': -1})
        
        # Fill nans in time with dummy times so that time operations work nicely
        var_stacked.time.attrs['calendar_type'] = 'Proleptic_Gregorian'
        var_stacked = var_stacked.assign_coords({
            'time': var_stacked.time.fillna(cftime.DatetimeProlepticGregorian(1800, 1, 1))})

        # Save
        my.to_zarr(var_region, f'./data/awap_{v}_{r}_ts.zarr')
        my.to_zarr(var_stacked, f'./data/awap_{v}_{r}.zarr')

Extracting precip over region(s) ['AUS', 'TAS', 'MEL']...


# End

In [ ]:
cluster.close()
client.close()